# アノテーション

### ボードの識別

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
    MODE = 3
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson Nanoを認識")
    I2C_BUSNUM = 1
    MODE = 0
elif BOARD_NAME == "JETSON_ORIN":
    print("Jetson AGX Orinを認識")
    I2C_BUSNUM = 7
    MODE = 0
elif BOARD_NAME == "JETSON_ORIN_NANO":
    print("Jetson Orin Nanoを認識")
    I2C_BUSNUM = 7
    MODE = 0

In [ ]:
!echo "jetson" | sudo -S nvpmodel -m $MODE

In [ ]:
!echo "jetson" | sudo -S nvpmodel -q

走行データはcameraフォルダに録画されています。今度は、cameraフォルダのデータにアノテーションをおこないます。

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import re
import ipywidgets
from utils import preprocess
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
from jupyter_clickable_image_widget import ClickableImageWidget
from jetcam.utils import bgr8_to_jpeg
import cv2
import torchvision.transforms as transforms
from xy_dataset import XYDataset
import time
import threading
import torch
import torchvision

DIR = "./"

IMG_WIDTH = 224
IMG_HEIGHT = 224

LOAD_CATEGORIES = ['xy','speed']
SAVE_CATEGORIES = ['xy','speed']

LOAD_DATASETS = ['X','Y','Z']
SAVE_DATASETS = ['RE_A','RE_B','RE_C','RE_D','RE_E', 'RE_F']

LOAD_TASK = ['camera','train','run','interactive']
SAVE_TASK = ['camera','train']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in SAVE_DATASETS:
    for task in SAVE_TASK:
        datasets[name] = XYDataset(task + '/' + name, SAVE_CATEGORIES, TRANSFORMS, random_hflip=True)
        
dataset = datasets[SAVE_DATASETS[0]]

In [ ]:
l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
SLEEP = [50,100,200,300,400,500]
SKIP = [1,2,3,4,5]

In [ ]:
sleep_widget = ipywidgets.Dropdown(options=SLEEP, description='sleep(ms)', index=0)
skip_widget = ipywidgets.Dropdown(options=SKIP, description='skip(枚)', index=0)
picture_widget = ClickableImageWidget(width=224, height=224)
no_widget = ipywidgets.IntText(description='no')
x_widget = ipywidgets.IntText(description='data x')
y_widget = ipywidgets.IntText(description='data y')
speed_widget = ipywidgets.IntText(description='data speed')
ai_x_widget = ipywidgets.IntText(description='AI　x')
ai_y_widget = ipywidgets.IntText(description='AI　y')
ai_speed_widget = ipywidgets.IntText(description='AI speed')
model_widget = ipywidgets.Text(description='model')
model_widget.value = "model.pth"
load_model_button = ipywidgets.Button(description='load model')
speed_slider = ipywidgets.IntSlider(description='speed', min=0, max=224, step=1, value=0, orientation='vertical')
add_speed_button = ipywidgets.Button(description='Add')

In [ ]:
device = torch.device('cuda')

output_dim = 2*len(LOAD_CATEGORIES)
#output_dim = 2
    
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)
model = model.to(device)

In [ ]:
def load_model(c):
    global load_model_widget, model, device
    
    model_name = load_model_widget.value
    if model_name != "[new]":
        device = torch.device('cuda')
        model = model.to(device)
        model.load_state_dict(torch.load(model_name))
        write_log(model_name + "のモデルを【読込】ました。")
    else:
        write_log("[new]が選択されているので、モデルは読み込めません。")

In [ ]:
def save_model(c):
    global save_model_name_widget, model, device
    path = "./model/"
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    torch.save(model.state_dict(), path + save_model_name_widget.value)
    write_log(path + save_model_name_widget.value + "に【保存】しました。")

In [ ]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget, save_task_widget, save_datasets_widget
    
    dataset = XYDataset(save_task_widget.value + '/' + save_datasets_widget.value, SAVE_CATEGORIES, TRANSFORMS, random_hflip=True)
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        #state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    #state_widget.value = 'live'

train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

display(train_eval_widget)

In [ ]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    stop = json_str["pwm_speed"]["stop"]
    left = json_str["pwm_steering"]["left"]
    center = json_str["pwm_steering"]["center"]
    right = json_str["pwm_steering"]["right"]

if stop == 0:
    INITIAL_VALUE=400
else:
    INITIAL_VALUE=stop
    
SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

STEERING_CH = 0
THROTTLE_CH = 1    
PCA9685.set_channel_value(STEERING_CH, center)
PCA9685.set_channel_value(THROTTLE_CH, stop)

In [ ]:
def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

def handle(x):
    x = map_rc(x, 224, 0, right, left)
    PCA9685.set_channel_value(STEERING_CH, x)

In [ ]:
from os.path import join
import subprocess
import datetime
import glob

def load_img(no):
    global running,no_widget,load_task_widget,load_category_widget,load_datasets_widget,img,load_flag,filenames
    load_task_value = load_task_widget.value
    category_value = load_category_widget.value
    datasets_value = load_datasets_widget.value
    
    xy_path = DIR + load_task_value + "/" + datasets_value + "/xy"
    speed_path = DIR + load_task_value + "/" + datasets_value + "/speed"
        
    xy_filenames = os.listdir(xy_path)
    xy_filenames = [join(xy_path, file_name) for file_name in xy_filenames]
    xy_imagenames = []

    for f in sorted(xy_filenames, key=os.path.getmtime):
        if ".jpg" == os.path.splitext(f)[1]:
            xy_imagenames.append(f)
            
    if no >= len(xy_imagenames):
        no_widget.value = no - 1
        write_log("ファイルが存在しません。" + str(len(xy_filenames)-1) + "以内の値を設定してください。")
        running = False
        return
        
    xy_name = xy_imagenames[no]
    
    pattern = '.*/(\d+)_(\d+)_.*'
    xy_result = re.match(pattern, xy_name)
    if xy_result:
        x = xy_result.group(1)
        y = xy_result.group(2)
        x_widget.value = x
        y_widget.value = y

    try:
        speed_filenames = os.listdir(speed_path)
        speed_filenames = sorted(speed_filenames)
        speed_imagenames = []
        for i in range(len(speed_filenames)):
            if ".jpg" == os.path.splitext(speed_filenames[i])[1]:
                speed_imagenames.append(speed_filenames[i]) 
                
        speed_name = speed_imagenames[no]
        speed_result = re.match(pattern, speed_name)
        if speed_result:
            speed = speed_result.group(2)
            speed_widget.value = speed
            speed_slider.value = speed
    except:
        speed_widget.value = 0
    img = cv2.imread(xy_name)
    marked_img = img.copy()
    
    if x != 0 or y != 0:
        marked_img = cv2.circle(marked_img, (int(x), int(y)), 8, (0, 255, 0), 3)
    
    try:
        preprocessed = preprocess(img)
        output = model(preprocessed).detach().cpu().numpy().flatten()
        result_x = output[0]
        result_y = output[1]
        result_speed = output[3]
        result_x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
        result_y = int(IMG_HEIGHT * (result_y / 2.0 + 0.5))
        result_speed = int(IMG_HEIGHT * (result_speed / 2.0 + 0.5))
        
        handle(result_x)
        ai_x_widget.value = result_x
        ai_y_widget.value = result_y
        ai_speed_widget.value = result_speed
        
        marked_img = cv2.circle(marked_img, (int(result_x), int(result_y)), 8, (255, 0, 0), 3)
        marked_img = cv2.line(marked_img,(112,0),(112,224),(255,255,0),1)
        marked_img = cv2.line(marked_img,(82,0),(82,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(52,0),(52,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(22,0),(22,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(142,0),(142,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(172,0),(172,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(202,0),(202,224),(100,100,255),1)

        marked_img = cv2.line(marked_img,(0,112),(224,112),(255,255,0),1)
        marked_img = cv2.line(marked_img,(0,82),(224,82),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,52),(224,52),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,22),(224,22),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,142),(224,142),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,172),(224,172),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,202),(224,202),(100,100,255),1)
        
        # Speed
        if result_speed> 224:
            result_speed = 244
        elif result_speed < 0:
            result_speed = 0
        marked_img = cv2.line(marked_img,(218,0),(218,224),(0,0,0),5)
        marked_img = cv2.line(marked_img,(219,224-result_speed),(219,224),(0,140,255),3)
        marked_img = cv2.putText(marked_img,"speed:"+str(result_speed),(160,215),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255))

    except:
        write_log("推論結果の反映に失敗しました。")

    picture_widget.value = bgr8_to_jpeg(marked_img)
    write_log(str(no) + "枚目の" + xy_name + "を読込ました。")

def del_pic(c):
    global load_category_widget,load_datasets_widget, filenames
    datasets_value = load_datasets_widget.value
    no = no_widget.value
    name = filenames[no]
    xy_path = DIR + TASK + "_" + datasets_value + "/xy/"  + name
    os.remove(xy_path)
    speed_path = DIR + TASK + "_" + datasets_value + "/speed/"  + name
    os.remove(speed_path)
    write_log(path + "を削除しました。")
    
def load_pic(c):
    global img,load_flag
    load_flag = True
    no = no_widget.value
    load_img(no)

def next_pic(c):
    global x,y,load_flag,skip
    load_flag = True
    no = no_widget.value
    no = int(no) + skip_widget.value
    no_widget.value = no
    load_img(no)
    
def before_pic(c):
    global x,y,load_flag,skip
    load_flag = True
    no = no_widget.value
    no = int(no) - skip_widget.value
    no_widget.value = no
    load_img(no)

def file_count():
    global datasets_xy_count_widget, datasets_speed_count_widget
    
    try:
        current_path = os.getcwd()
        xy_path = current_path + "/" + save_task_widget.value + "/" + save_datasets_widget.value + "/xy/"
        speed_path = current_path + "/" + save_task_widget.value + "/" + save_datasets_widget.value + "/speed/"
        
        xy_is_file = os.path.isdir(xy_path)
        
        if xy_is_file:
            xy_file_count = sum(os.path.isfile(os.path.join(xy_path,name)) for name in os.listdir(xy_path))
            datasets_xy_count_widget.value = xy_file_count
        
        speed_is_file = os.path.isdir(speed_path)
        
        if speed_is_file:
            speed_file_count = sum(os.path.isfile(os.path.join(speed_path,name)) for name in os.listdir(speed_path))
            datasets_speed_count_widget.value = speed_file_count
            
        if xy_is_file == False and speed_is_file == False:
            datasets_xy_count_widget.value = 0
            datasets_speed_count_widget.value = 0
            
    except:
        datasets_xy_count_widget.value = 0
        datasets_speed_count_widget.value = 0
    
    
    
def save_snapshot(_, content, msg):
    global img,x,y,load_flag,save_datasets_widget,save_task_widget,save_category_widget
    if content['event'] == 'click' and load_flag == True:
        load_flag = False
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']

        remarked_img = img.copy()
        remarked_img = cv2.circle(remarked_img, (int(x), int(y)), 8, (0, 255, 0), 3)
        picture_widget.value = bgr8_to_jpeg(remarked_img)
        name = save_datasets_widget.value
        write_log("["+save_task_widget.value + "/" + name + "]の" + SAVE_CATEGORIES[0] + "カテゴリにデータを追加しました。")
        dataset = datasets[name]
        dataset.save_entry("xy", img, x, y)
        #write_log("新しい座標で保存しました。")
        
        file_count()
        
        
        
def save_speed(c):
    global img,speed_slider,save_datasets_widget,save_task_widget
    speed = speed_slider.value
    remarked_img = img.copy()
    name = save_datasets_widget.value
    dataset = datasets[name]
    dataset.save_entry("speed", img, 0, speed)
    write_log("["+save_task_widget.value + "/" + name + "]の" + SAVE_CATEGORIES[1] + "カテゴリにデータを追加しました。")
    
    file_count()
        
def live():
    global running, skip, sleep_time
    load_flag = True
    no = no_widget.value
    while running:
        no += skip
        no_widget.value = no
        try:
            load_img(no)
        except:
            write_log("no: " + no + "のファイルの読込に失敗")
        time.sleep(sleep_time/1000)  
    
def play(c):
    global running, execute_thread, skip, sleep_time, skip_widget, sleep_widget
    skip = skip_widget.value
    sleep_time = sleep_widget.value
    running = True
    execute_thread = threading.Thread(target=live)
    execute_thread.start()
    
def stop(c):
    global running, execute_thread
    running = False
    execute_thread.join()
    write_log("STOP")
        
picture_widget.on_msg(save_snapshot)

play_button = ipywidgets.Button(description='play')
stop_button = ipywidgets.Button(description='stop')

save_model_button = ipywidgets.Button(description='save model')

load_pic_button = ipywidgets.Button(description='load image')
next_pic_button = ipywidgets.Button(description='>')
before_pic_button = ipywidgets.Button(description='<')
del_pic_button = ipywidgets.Button(description='delete image')

load_model_widget = ipywidgets.Dropdown(options=[],description='読込モデル')
load_model_time_widget = ipywidgets.Text(description='作成日時')
save_model_name_widget = ipywidgets.Text(description='保存モデル名',value="model.pth")

play_button.on_click(play)
stop_button.on_click(stop)

add_speed_button.on_click(save_speed)

load_model_button.on_click(load_model)
save_model_button.on_click(save_model)
load_pic_button.on_click(load_pic)
next_pic_button.on_click(next_pic)
before_pic_button.on_click(before_pic)
del_pic_button.on_click(del_pic)

load_datasets_widget = ipywidgets.Dropdown(options=LOAD_DATASETS, description='dataset', index=0)
save_datasets_widget = ipywidgets.Dropdown(options=SAVE_DATASETS, description='dataset')
datasets_xy_count_widget = ipywidgets.IntText(description='XY Count')
datasets_speed_count_widget = ipywidgets.IntText(description='Speed Count')

def set_dataset(change):
    global dataset
    datasets[change['new']] = XYDataset(save_task_widget.value + '/' + change['new'], SAVE_CATEGORIES, TRANSFORMS, random_hflip=True)
    #dataset = datasets[change['new']]
    #write_log(change['new'])
save_datasets_widget.observe(set_dataset, names='value')

load_task_widget = ipywidgets.Dropdown(options=LOAD_TASK, description='task')
save_task_widget = ipywidgets.Dropdown(options=SAVE_TASK,  value=SAVE_TASK[1], description='task')

def change_load_task(change):
    global dataset, DIR, load_task_widget, load_datasets_widget
    try:
        path = DIR + load_task_widget.value + "/"
        files = os.listdir(path)
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = [f for f in files if f != ".ipynb_checkpoints"]
        dirs = sorted(dirs)
        load_datasets_widget.options = dirs
    except:
        write_log(path + "が存在していません。")
        load_datasets_widget.options = []
load_task_widget.observe(change_load_task, names='value')
change_load_task(LOAD_TASK[0])

def change_save_task(change):
    global dataset, DIR, save_task_widget, save_datasets_widget
    try:
        path = DIR + save_task_widget.value + "/"
        if not os.path.exists(path):
            subprocess.call(['mkdir', '-p', path])
        #files = os.listdir(path)            
        #dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        #dirs = [f for f in files if f != ".ipynb_checkpoints"]
        #dirs = sorted(dirs)
        #save_datasets_widget.options = dirs
        save_datasets_widget.options = SAVE_DATASETS
    except:
        write_log(path + "が存在していません。")
        save_datasets_widget.options = []
save_task_widget.observe(change_save_task, names='value')
change_save_task(SAVE_TASK[1])

def change_save_dataset(change):
    global dataset, DIR, save_task_widget, save_datasets_widget
    file_count()
save_datasets_widget.observe(change_save_dataset, names='value')
change_save_dataset(SAVE_DATASETS[0])

def change_sleep(change):
    global sleep_time, sleep_widget
    sleep_time = sleep_widget.value
sleep_widget.observe(change_sleep, names='value')

def change_skip(change):
    global sleep, sleep_widget
    skip = skip_widget.value
skip_widget.observe(change_skip, names='value')

def model_list(change):
    global load_model_widget
    try:
        files = glob.glob('./model/*.pth', recursive=True)
        files.insert(0,"[new]")
        load_model_widget.options = files
        load_model_time_widget.value = ""
    except:
        load_model_widget.options = []
model_list("list")

def change_file(change):
    global load_model_widget
    try:
        file = load_model_widget.value
        ts = os.path.getctime(file)
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        load_model_time_widget.value = s
    except:
        load_model_time_widget.value = ""
load_model_widget.observe(change_file, names='value')


load_category_widget = ipywidgets.Dropdown(options=LOAD_CATEGORIES, description='category')
save_category_widget = ipywidgets.Dropdown(options=SAVE_CATEGORIES, description='category')

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([Label('【load】'),load_datasets_widget,load_task_widget]),
    ipywidgets.HBox([Label('【save】'),ipywidgets.VBox([save_datasets_widget,datasets_xy_count_widget,datasets_speed_count_widget]),save_task_widget]),
    ipywidgets.HBox([no_widget,load_pic_button,del_pic_button]),
    ipywidgets.HBox([play_button,stop_button,before_pic_button,next_pic_button]),
    ipywidgets.HBox([sleep_widget, skip_widget]),
    ipywidgets.HBox([picture_widget,ipywidgets.VBox([speed_slider,add_speed_button]),ipywidgets.VBox([x_widget,y_widget,speed_widget,ai_x_widget,ai_y_widget,ai_speed_widget])]),
    ipywidgets.HBox([epochs_widget,train_button,eval_button]),
    ipywidgets.HBox([load_model_widget,load_model_time_widget,load_model_button]),
    ipywidgets.HBox([progress_widget,loss_widget]),
    ipywidgets.HBox([save_model_name_widget,save_model_button]),
    process_widget,
])
display(data_collection_widget)
